In [56]:

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

import xgboost as xgb

import gradio as gr

In [57]:
# Load the training data from a CSV file
train = pd.read_csv('train.csv')

# Load the test data from a CSV file
test = pd.read_csv('test.csv')

# Print the shape of the training data
# This will output a tuple (number of rows, number of columns)
print(train.shape)

(1460, 81)


In [58]:
# Display the first 5 rows of the training data
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [59]:
# Drop the 'Id' column from both the training and test dataframes
train = train.drop(['Id'], axis=1)
test = test.drop(['Id'], axis=1)

In [60]:
# Plot the distribution of SalePrice using seaborn and matplotlib
plt.figure(figsize=(9, 8))
sns.distplot(train['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});

C:\Users\tmsol\AppData\Local\Temp\ipykernel_23252\3662643944.py:3: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(train['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN be

In [61]:
# Print a list of unique data types in the 'train' dataframe
list(set(train.dtypes.tolist()))

print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [62]:
# Select only the numerical columns from the training data
df_num = train.select_dtypes(include = ['float64', 'int64'])

# Select only the numerical columns from the test data
df_numT = test.select_dtypes(include = ['float64', 'int64'])

# Display the first 5 rows of the numerical training data
df_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000


In [63]:
# Plot histograms for each numerical column in the training data
train.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [64]:
# Separate the target variable ('SalePrice') from the predictors
X_train, y_train = train.drop(['SalePrice'], axis=1), train['SalePrice']

# Import the LabelEncoder class from sklearn
from sklearn.preprocessing import LabelEncoder

# Get a list of categorical columns
# This selects all columns of type 'object', which are the categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a LabelEncoder object
le = LabelEncoder()

# Encode the categorical columns
# This converts each category to a unique integer
for col in categorical_cols:
    X_train[col] = le.fit_transform(X_train[col].astype(str))

# Create a DMatrix object from the training data
# This is a data structure optimized for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if

In [65]:
# Define the parameters for the XGBoost model
params = {
    'objective':'reg:squarederror',
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'learning_rate': 0.01,
    'alpha': 10,
    'lambda': 1,
}

# Define the number of boosting rounds
num_round = 1000

# Train the XGBoost model
bst = xgb.train(params, dtrain, num_round)

# Encode the categorical columns in the test data
for col in categorical_cols:
    test[col] = le.fit_transform(test[col].astype(str))

# Create a DMatrix object from the test data
dtest = xgb.DMatrix(test, enable_categorical=True)

# Use the trained model to make predictions on the test data
preds = bst.predict(dtest)

# Create a DataFrame with the predictions and the corresponding IDs
output = pd.DataFrame({'Id': test.index+1461, 'SalePrice': preds})

# Save the predictions to a CSV file
output.to_csv('output.csv', index=False)

# Print a success message
print("Your submission was successfully saved!")

c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if

Your submission was successfully saved!


In [66]:
# Define a function to predict house price
def predict_house_price(lotArea, LotFrontage, OverallQual, OverallCond, YearBuilt, YearRemodAdd):
    # Initialize a dictionary to hold the mean values of the features
    mean_values = {}

    # For each column in the training data
    for col in train.columns:
        # If the column is categorical
        if train[col].dtype == 'object':
            # Use the mode (most frequent value) for the mean value
            mean_values[col] = train[col].mode()[0]
        else:  # If the column is numerical
            # Use the mean for the mean value
            mean_values[col] = train[col].mean()

    # Set the mean values for the features we're predicting on
    mean_values['LotFrontage'] = LotFrontage
    mean_values['LotArea'] = lotArea
    mean_values['OverallQual'] = OverallQual
    mean_values['OverallCond'] = OverallCond
    mean_values['YearBuilt'] = YearBuilt
    mean_values['YearRemodAdd'] = YearRemodAdd

    # Encode the categorical columns in the test data
    for col in categorical_cols:
        mean_values[col] = le.fit_transform([mean_values[col]])

    # Create a DataFrame from the mean values dictionary
    dtest = pd.DataFrame(mean_values, index=[0])

    # Check if 'SalePrice' and 'Id' columns exist in the DataFrame and drop them if they do
    # These columns are not needed for the prediction
    if 'SalePrice' in dtest.columns:
        dtest = dtest.drop(['SalePrice'], axis=1)
    if 'Id' in dtest.columns:
        dtest = dtest.drop(['Id'], axis=1)

    # Convert the DataFrame into a DMatrix object
    # DMatrix is a data structure used by XGBoost that is optimized for both memory efficiency and training speed
    dtest = xgb.DMatrix(dtest, enable_categorical=True)

    # Use the trained XGBoost model to make predictions on the test data
    preds = bst.predict(dtest)

    # Return the first prediction
    # Since we're only predicting on one sample, there's only one prediction
    return preds[0]
        

    
   



# Create a gradio Interface
iface = gr.Interface(
    # The function to predict house price
    fn=predict_house_price,
    # The inputs to the function
    inputs=[
        gr.components.Slider(0, 100000, label="Lot Area"),  # Slider for inputting Lot Area
        gr.components.Slider(0, 100, label="Lot Frontage"),  # Slider for inputting Lot Frontage
        gr.components.Slider(0, 10, label="Overall Quality"),  # Slider for inputting Overall Quality
        gr.components.Slider(0, 10, label="Overall Condition"),  # Slider for inputting Overall Condition
        gr.components.Slider(1900, 2020, label="Year Built"),  # Slider for inputting Year Built
        gr.components.Slider(1900, 2020, label="Year Remod Add")  # Slider for inputting Year Remod Add
    ],
    # The type of the output
    outputs="number",
    # The title of the Interface
    title="House Price Predictor",
    # The description of the Interface
    description="Predicts the price of a house given some of its features."
)

# Launch the Interface
# The 'share' parameter is set to True, which means the Interface will be publicly accessible
iface.launch(share=True)

    
    
    

Running on local URL:  http://127.0.0.1:7898
Running on public URL: https://eb1f42730dfc6e8515.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [67]:
# Define a function to predict house price
def predict_house_price(lotArea, LotFrontage, OverallQual, OverallCond, YearBuilt, YearRemodAdd):
    # Initialize a dictionary to hold the mean values of the features
    mean_values = {}

    # For each column in the training data
    for col in train.columns:
        # If the column is categorical
        if train[col].dtype == 'object':
            # Use the mode (most frequent value) for the mean value
            mean_values[col] = train[col].mode()[0]
        else:  # If the column is numerical
            # Use the mean for the mean value
            mean_values[col] = train[col].mean()

    # Set the mean values for the features we're predicting on
    mean_values['LotFrontage'] = LotFrontage
    mean_values['LotArea'] = lotArea
    mean_values['OverallQual'] = OverallQual
    mean_values['OverallCond'] = OverallCond
    mean_values['YearBuilt'] = YearBuilt
    mean_values['YearRemodAdd'] = YearRemodAdd

    # Encode the categorical columns in the test data
    for col in categorical_cols:
        mean_values[col] = le.fit_transform([mean_values[col]])

c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\xgboost\data.py:299: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\xgboost\data.py:301: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\xgboost\data.py:332: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
c:\Users\tmsol\anaconda3\envs\MLass2\Lib\site-packages\xgboost\data.py:323: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int or is_bool or is_